## HW2: Linear Discriminant Analysis
In hw2, you need to implement [Fisher’s linear discriminant](https://towardsdatascience.com/fishers-linear-discriminant-intuitively-explained-52a1ba79e1bb) by using only numpy, then train your implemented model by the provided dataset and test the performance with testing data

Please note that only **NUMPY** can be used to implement your model, you will get **zero point** by simply calling `sklearn.discriminant_analysis.LinearDiscriminantAnalysis` 

## Load data

In [1]:
import numpy as np

In [2]:
x_train, x_test, y_train, y_test = np.load('classification_data.npy', allow_pickle=True)

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3750, 2)
(3750,)
(1250, 2)
(1250,)


## 1. Compute the mean vectors mi, (i=1,2) of each 2 classes

In [4]:
## Your code HERE

m1 = np.zeros((1,2))
m2 = np.zeros((1,2))
count1 = 0.0
count2 = 0.0
size = x_train.size / x_train[0].size
for i in range(int(size)):
    if(y_train[i] == 0):
        m1[0,0] = m1[0,0] + x_train[i,0]
        m1[0,1] = m1[0,1] + x_train[i,1]
        count1 = count1+1.0
    else:
        m2[0,0] = m2[0,0]+x_train[i,0]
        m2[0,1] = m2[0,1]+x_train[i,1]
        count2 = count2+1.0
m1 = m1/count1
m2 = m2/count2

In [5]:
print(f"mean vector of class 1: {m1}", f"mean vector of class 2: {m2}")

mean vector of class 1: [[ 0.99253136 -0.99115481]] mean vector of class 2: [[-0.9888012   1.00522778]]


## 2. Compute the Within-class scatter matrix SW

In [6]:
## Your code HERE
sw = np.zeros((x_train[0].size,x_train[0].size))
for i in range(0, int(size)):
    if(y_train[i] == 0):
        sw = sw + (x_train[i]-m1).T@(x_train[i]-m1)
    else:
        sw = sw + (x_train[i]-m2).T@(x_train[i]-m2)

In [7]:
print(f"Within-class scatter matrix SW: {sw}")

Within-class scatter matrix SW: [[ 4337.38546493 -1795.55656547]
 [-1795.55656547  2834.75834886]]


## 3.  Compute the Between-class scatter matrix SB

In [8]:
## Your code HERE
sb = (m2-m1).T @ (m2-m1)

In [9]:
print(f"Between-class scatter matrix SB: {sb}")

Between-class scatter matrix SB: [[ 3.92567873 -3.95549783]
 [-3.95549783  3.98554344]]


## 4. Compute the Fisher’s linear discriminant

In [10]:
## Your code HERE
vec = np.linalg.inv(sw)@sb
e, v = np.linalg.eig(vec)
maxx = np.argmax(e)
w = v[:,maxx:maxx+1].T

In [11]:
print(f" Fisher’s linear discriminant: {w}")

 Fisher’s linear discriminant: [[ 0.37003809 -0.92901658]]


## 5. Project the test data by linear discriminant and get the class prediction by `K` nearest-neighbor rule. Please report the accuracy score with `K` values from 1 to 5
you can use `accuracy_score` function from `sklearn.metric.accuracy_score`

In [12]:
import math
from sklearn.metrics import accuracy_score
w = w/math.sqrt(w[0,0] ** 2 + w[0,1] ** 2)
project_x = x_train@w.T
print(project_x)
def classfication(project_x,y,test,k):
    x = project_x
    index = np.argsort(np.abs(x - test),axis = 0)[:k]
    ans = y[index]
    if(ans.sum()>ans.size/2):
        return 1
    else:
        return 0
y_pred = np.zeros((1,y_test.size))    
project = x_test@w.T
for k in range(1,6):
    for i in range(0,project.size):
        y_pred[0,i] = classfication(project_x,y_train,project[i,0],k)
    # print(y_pred.shape)    
    acc = accuracy_score(y_test, y_pred.T)
    print(acc)

[[ 2.68116355]
 [-1.6046331 ]
 [-1.19503736]
 ...
 [ 1.02669749]
 [-1.22120151]
 [-1.23507452]]
0.8488
0.8704
0.8792
0.8824
0.8912


In [13]:
print(f"Accuracy of test-set {acc}")

Accuracy of test-set 0.8912


## 6. Plot the 1) best projection line on the training data and show the slope and intercept on the title (you can choose any value of intercept for better visualization) 2) colorize the data with each class 3) project all data points on your projection line. Your result should look like [this image](https://i.imgur.com/tubMQpw.jpg)